# Competing phases 

To calculate the limiting chemical potentials of elements in the material (needed for calculating the defect formation energies) we need to consider the energies of all competing phases. `doped` does this by calling the `CompetingPhases` Class, which then queries Materials Project to obtain all the relevant competing phases to be calculated.
In some cases the Materials Project may not have all known phases in a certain chemical space (e.g. in the La-Mn-O system discussed here), so it's a good idea to cross-check the generated competing phases with the [ICSD](https://icsd.products.fiz-karlsruhe.de/) in case you suspect any are missing.

For this functionality to work correctly, you must have POTCARs set up for [pymatgen](https://pymatgen.org/installation.html#potcar-setup) and you will also need your Materials Project [API key](https://legacy.materialsproject.org/dashboard) set up in `pymatgen`.
- Note that at present this uses the 'Legacy API' from the Materials Project, and so the API key you use (either in `~/.pmgrc.yaml` or supplied to `CompetingPhases` with the `api_key` parameter) should correspond to the Materials Project legacy API. This key can be found [here](https://legacy.materialsproject.org/dashboard).

Doped assumes the so-called "molecule in a box" structures for the gaseous elemental phases H$_2$, O$_2$, N$_2$, F$_2$ and Cl$_2$. The molecule is placed in a 30 Å x 30 Å x 30 Å box, and relaxed with Γ-point-only k-point sampling.

In [1]:
from doped.competing_phases import CompetingPhases

For example, if we were interested in the competing phases of ZrO2, we would search across the Zr-O system like so:

In [2]:
system = ['Zr', 'O']
cp = CompetingPhases(system, e_above_hull=0.03)
# If this returns a KeyError then it means you are using an API key for the new Materials Project API,
# and not the legacy API as required – see the bulletpoint above

`cp.competing_phases` contains all the competing phases, their structures, magnetic moment and (MP-calculated GGA) band gaps. We can check how many there are by: 

In [3]:
print(len(cp.competing_phases))

14


From there you can set up the competing phase calculations with `doped` as decribed below, or do your own thing with python / atomate / aiida to set up the calculations.

k-points convergence testing is done at GGA (PBEsol by default) and it set up to account for the magnetic moment convergence as well. All of this interfaces with [vaspup2.0](https://github.com/kavanase/vaspup2.0) so it's easy to use on the HPCs (with the `generate-converge` command to run the calculations and `data-converge` to quickly parse and analyse the results).
You may want to change the default `ENCUT` (480 eV) or k-point densities that the convergence tests span (5 - 60 kpoints/Å$^3$ for semiconductors & insulators and 40 - 120 kpoints/Å$^3$ for metals in steps of 5 kpoints/Å$^3$). Note that `ISMEAR = -5` is used for metals by default (better kpoint convergence for energies but should not be used during metal geometry relaxations) and k-point convergence testing is not required for molecules (Γ-point sampling is sufficient).

The kpoints convergence calculations are set up with:

In [5]:
cp.convergence_setup(user_incar_settings={'ENCUT':550})  # For custom INCAR settings, any flags that aren't numbers or True/False need to be input as strings with quotation marks

O2 is a molecule in a box, does not need convergence testing


This creates a folder called `competing_phases` with all the stable & unstable competing phases and the k-point convergence test calculation directories. These can be quickly run on HPCs using [vaspup2.0](https://github.com/kavanase/vaspup2.0), by creating a `job` file for the HPC scheduler (`vaspup2.0` example [here](https://github.com/kavanase/vaspup2.0/blob/master/input/job)), copying it into each directory and running the calculation with a `bash` loop like:

```bash
for i in *EaH*  # (in the competing_phases directory) – for each competing phase
do cp job $i
cd $i
for k in k*   # for each kpoint calculation directory
do cp job $k
cd $k
qsub job  # may need to change 'qsub' to 'sbatch' if the HPC scheduler is SLURM
cd ..
done
cd ..
done
```

Within each competing phase directory in `competing_phases`, the `vaspup2.0` `data-converge` command can be run to quickly parse the results and determine the converged _k_-mesh (see the [vaspup2.0](https://github.com/kavanase/vaspup2.0) homepage for examples).

Next, you want to relax each competing phase and calculate the energy with the same DFT functional and settings as your defect supercell calculations. `doped` can generate these folders for the relaxation of the competing phases.

The _k_-point meshes are gamma-centred (as opposed to Monkhorst-Pack) by default. By default `doped` will make the inputs assuming a HSE06 `INCAR` (see `[HSE06_config_relax.json](https://github.com/SMTG-UCL/doped/blob/master/doped/HSE06_config_relax.json)` for default values) and kpoint densities of 95 kpoints/Å$^3$ for metals and 45 kpoints/Å$^3$ for semiconductors. Assuming you've followed the k-point convergence testing workflow above, you should change the `KPOINTS` file to match the converged mesh in each case, however the default densities are good starting points. `doped` will automatically set `SIGMA` and `ISMEAR` accordingly depending on whether the phase is a semiconductor or metal, and will set `NUPDOWN` appropriately for molecules (i.e. O$_2$ has triplet spin).

These relaxations can be set up with:

In [6]:
cp.vasp_std_setup(user_incar_settings={'ENCUT':720})  # For custom INCAR settings, any flags that aren't numbers or True/False need to be input as strings with quotation marks

Remember that the final `ENCUT` used for the energy calculations should be the same as for your host material & defects, and that you may still need to account for Pulay stress by increasing `ENCUT` for the geometry relaxations (usual rule of thumb being 1.3*converged `ENCUT`) or re-relaxing each structure until the volume change is minimal (roughly <0.3%).

## Additional competing phases

So you've done your intrinsic defects and now you want to consider extrinsic doping. The addition of the new extrinsic species can also be handled with doped by the aptly named `AdditionalCompetingPhases` class. 

In [4]:
from doped.competing_phases import AdditionalCompetingPhases

In [17]:
system = ['Zr', 'O']
extrinsic_species = 'La'
acp = AdditionalCompetingPhases(system, extrinsic_species, e_above_hull=0.03)

Doped can very cleverly tell what phases you've already calculated before and which ones should be added anew so it limits the number as we can see here:

In [6]:
len(acp.competing_phases)

29

The set up for convergence testing and relaxations is done in the exact same way as before: 

In [ ]:
acp.convergence_setup(user_incar_settings={'ENCUT':550})  # For custom INCAR settings, any flags that aren't numbers or True/False need to be input as strings with quotation marks

In [11]:
acp.vasp_std_setup(user_incar_settings={'ENCUT':720})  # For custom INCAR settings, any flags that aren't numbers or True/False need to be input as strings with quotation marks

# Competing phases analyzer 


In [9]:
from doped.competing_phases import CompetingPhasesAnalyzer

## Read in data from `vasprun.xml`

Once you've calculated all your very many competing phases you will want to analyse them. First you will want all your vaspruns neatly organised in some tree structure. To get them all off the HPCs recursively without including any other large files you can recursively rsync: 

```bash 
rsync -azvuR hpc:'path/to/the/base/folder/competing_phases/./*_EaH_*/vasp_std/vasprun.xml' . 
```

where the `/./` indicates where you'd like to start the recurse from so you only keep the folder structure from the `formula_EaH_*` onwards. If you've done SOC calculations obviously change vasp_std to vasp_ncl or whatever you've called the folders. Optionally `gzip vasprun.xml` the files first, but remember to change the filename in the rsync command

All analysis is done with aptly named `CompetingPhasesAnalyzer` and if you've used doped all you need to supply it is the 'pretty' formula of the system you're solving the chemical limits for (in this case that would be `'ZrO2'` and the path to the base folder in which you have all your `formula_EaH_*/vasp_std/vasprun.xml` in. If you've used `vasp_ncl` (or anything else) instead of `vasp_std` you can set that as well. 

If you've not generated your competing phases inputs with doped, worry not because we've accounted for that too. You can generate a list of paths (or strings) to the vaspruns from using `pathlib` or `os`. 


In [27]:
system = 'ZrO2'
cpa = CompetingPhasesAnalyzer(system)

In [28]:
cpa.from_vaspruns(path='./competing_phases/ZrO/', 
                  folder='relax', 
                  csv_fname='competing_phases/zro2_competing_phase_energies.csv')

Can't find a vasprun.xml(.gz) file for competing_phases/ZrO/.DS_Store, proceed with caution
parsing 8 vaspruns, this may take a while


The read in from vaspruns only needs to be done once, as the energies are saved to a csv file. 

An example of how to get all the vaspruns in one list if you've not used doped to generate them: 

In [ ]:
from pathlib import Path 
path = 'path/to/base'
all_paths = []
for p in path.iterdir():
    if not p.name.startswith('.'): 
        pp = p / 'relax' / 'vasprun.xml' 
        if pp.is_file():
            all_paths.append(pp)

## Read in data from a csv

As a sidenote you can also read in the data from a csv, as long as it contains the following headers: `'formula', 'energy_per_fu', 'energy', 'formation_energy'` 


In [29]:
cpa.from_csv('competing_phases/zro2_competing_phase_energies.csv')

## Calculate the chemical potential limits

As easy as: 

In [30]:
cpa.calculate_chempots(csv_fname='competing_phases/zro2_chempots.csv')

Calculated chemical potential limits: 

          Zr         O
0 -10.975428  0.000000
1  -0.199544 -5.387942


This should save your chempots to a csv and also print them out for your viewing pleasure.

To get the `doped.formation_energy_table`-compliant input, use: 

In [15]:
cpa.intrinsic_chem_limits 

{'facets': {'ZrO2-O2': {Element Zr: -20.81910468, Element O: -7.006602065},
  'Zr3O-ZrO2': {Element Zr: -10.04321996, Element O: -12.394544425}},
 'elemental_refs': {Element O: -7.006602065, Element Zr: -9.84367624},
 'facets_wrt_el_refs': {'ZrO2-O2': {Element Zr: -10.975428439999998,
   Element O: 0.0},
  'Zr3O-ZrO2': {Element Zr: -0.19954371999999942,
   Element O: -5.387942359999999}}}

## Introducing extrinsic species / dopants 

If you're dealing with dopants in your defect system, you will also want to consider the dopant competing phases and add them to the phase diagram. Assuming you've added them using `AdditionalCompetingPhases`, you can easily parse them using `CompetingPhaseAnalyzer`.

In [21]:
system = 'ZrO2' 
extrinsic_species='La'
cpa = CompetingPhasesAnalyzer(system, extrinsic_species=extrinsic_species)
cpa.from_vaspruns(path='./competing_phases/LaZrO/', 
                  folder='relax', 
                  csv_fname='./competing_phases/zro2_la_competing_phase_energies.csv')
cpa.calculate_chempots(csv_fname='./competing_phases/zro2_la_chempots.csv')

Can't find a vasprun.xml(.gz) file for competing_phases/LaZrO/.DS_Store, proceed with caution
parsing 11 vaspruns, this may take a while
Calculating chempots for La
Calculated chemical potential limits: 

          Zr         O        La La_limiting_phase
0 -10.975428  0.000000 -9.462987          La2Zr2O7
1  -0.199544 -5.387942 -1.381074          La2Zr2O7


The chemical potential of the extrinsic species is calculated at each of the existing intrinsic chemical potentials. This assumes that the extrinsic species chemical potential is restricted by the host (intrinsic) material chemical potential. 

This is in line with `pyCDT`'s `full_sub_approach=False` but it actually works. 

To get the chempots needed for doped defects parsing you need to run the above once and then you can get them from `cpa.chem_limits` for extrinsic or `cpa.intrinsic_chem_limits` for intrinsic. Because they are a dict you can easily pickle them like usual 

### CPLAP input

If you don't trust doped and pymatgen (ye of little faith), you can also create the `input.dat` file to use with [CPLAP](https://github.com/jbuckeridge/cplap). You can set the dependent variable, or leave it to doped to decide which one it will pick (all should yield the same numbers in higher order systems)

In [ ]:
cpa.cplap_input(dependent_variable='O') 

### Visualising the chemical potential limits 

For higher order systems, this interfaces really well with pymatgen's three-dimensional plotters, the list of total energies for the `ChemicalPotentialDiagram` are accessible from `cpa.pd_energies` and you can get the elemental energies to subtract from the `cpd.domains` by looping over `cpa.pd_energies` to find the ones with elemental formulas